In [1]:
import pandas as pd
from sqlalchemy import create_engine,text
import pymysql.cursors
import os
# import getpass
import urllib.parse

In [2]:
pw_raw = os.getenv('mysql_xt')
pw = urllib.parse.quote_plus(pw_raw)

# python --> mysql

In [ ]:
connection_string = 'mysql+pymysql://root:' + pw + '@127.0.0.1:3306/'
engine = create_engine(connection_string)

In [ ]:
with engine.connect() as conn:
    conn.execute(text("CREATE DATABASE IF NOT EXISTS project_ab_testing"))

In [ ]:
client_prof=pd.read_csv('df_final_demo.txt')
client_prof.to_sql('client_profile',engine, 'project_ab_testing', if_exists='replace', index=False)

In [ ]:
experiment_roster=pd.read_csv('df_final_experiment_clients.txt')
experiment_roster.to_sql('experiment_roster',engine, 'project_ab_testing', if_exists='replace', index=False)

In [ ]:
experiment_roster_no_null=experiment_roster.dropna()
experiment_roster_no_null.to_sql('experiment_roster_no_null',engine, 'project_ab_testing', if_exists='replace', index=False)

In [ ]:
pt1=pd.read_csv('df_final_web_data_pt_1.txt')
pt2=pd.read_csv('df_final_web_data_pt_2.txt')
digital_footprints = pd.concat([pt1,pt2])
digital_footprints = digital_footprints[~digital_footprints.duplicated()]
digital_footprints.to_sql('digital_footprints',engine, 'project_ab_testing', if_exists='replace', index=False)

In [ ]:
df_with_variation = digital_footprints.merge(experiment_roster, on='client_id', how='left').sort_values(by="date_time")
test_group = df_with_variation[df_with_variation['Variation'] == 'Test']
test_group.to_sql('test_group',engine, 'project_ab_testing', if_exists='replace', index=False)
control_group = df_with_variation[df_with_variation['Variation'] == 'Control']
control_group.to_sql('control_group',engine, 'project_ab_testing', if_exists='replace', index=False)

In [ ]:
summary_table=pd.read_csv('summary_table.csv')
summary_table.to_sql('summary_table',engine, 'project_ab_testing', if_exists='replace', index=False)

In [ ]:
test_group=pd.read_csv('test_group.csv')
test_group.to_sql('test_group',engine, 'project_ab_testing', if_exists='replace', index=False)
control_group=pd.read_csv('control_group.csv')
control_group.to_sql('control_group',engine, 'project_ab_testing', if_exists='replace', index=False)

# mysql -- > python

In [3]:
db='project_ab_testing'
connection_string = 'mysql+pymysql://root:' + pw + '@127.0.0.1:3306/'+db
engine = create_engine(connection_string)

In [4]:
with engine.connect() as connection:
    query = text('SELECT * FROM df_with_variation_clean')
    result = connection.execute(query)
    df_with_variation_clean = pd.DataFrame(result.all())

In [5]:
df_with_variation_clean.nunique()

client_id       50500
visitor_id      56011
visit_id        69205
process_step        5
date_ymd           98
time_stamp      69704
variation           2
dtype: int64

In [6]:
with engine.connect() as connection:
    query = text('SELECT * FROM each_step_count')
    result = connection.execute(query)
    each_step_count = pd.DataFrame(result.all())

In [ ]:
each_step_count  # here visit_id all unique --> 69205 rows

In [7]:
with engine.connect() as connection:
    query = text('SELECT * FROM number_visits_per_client')
    result = connection.execute(query)
    number_visits_per_client = pd.DataFrame(result.all())

In [ ]:
number_visits_per_client

In [8]:
with engine.connect() as connection:
    query = text('SELECT * FROM last_process_step')
    result = connection.execute(query)
    last_process_step = pd.DataFrame(result.all())

In [ ]:
last_process_step

In [9]:
all_data_for_step = pd.merge(each_step_count,last_process_step,on=['visit_id','visitor_id'],how='left')

In [ ]:
all_data_for_step #.sort_values('client_id')

In [ ]:
all_data_for_step.nunique()

In [10]:
all_visit_data=pd.merge(number_visits_per_client,all_data_for_step,on=['client_id', 'variation'],how='inner').sort_values('client_id')

In [11]:
# add column to check if last_step is "confirm"
all_visit_data['last_step_confirm'] = all_visit_data['last_process_step'].apply(lambda x: 1 if x == 'confirm' else 0)

In [12]:
# add column "step_hierarchy"
def highest_step(row):
    if row['confirm_count'] > 0:
        return 'confirm'
    elif row['step_2_count'] > 0:
        return 'step_2'
    elif row['step_1_count'] > 0:
        return 'step_1'
    elif row['start_count'] > 0:
        return 'start'
    else:
        return None

all_visit_data['step_hierarchy'] = all_visit_data.apply(highest_step, axis=1)

In [13]:
# add column to check if step_hierarchy is "confirm", which means task completed 
all_visit_data['if_confirm'] = all_visit_data['step_hierarchy'].apply(lambda x: 1 if x == 'confirm' else 0)

In [ ]:
all_visit_data #.head(50)

In [14]:
# export test_group frame to CSV

# Get the current working directory
current_directory = os.getcwd()

# Specify the file name
file_name = "all_visit_steps.csv"

# Join the current directory with the file name to create the file path
file_path = os.path.join(current_directory, file_name)

# Export the summary_table DataFrame to a CSV file
all_visit_data.to_csv(file_path, index=False)

print(f"Test group table has been exported to {file_path}")

Test group table has been exported to c:\Users\54189\ironhack_262\project2\project_ab_testing sql\all_visit_steps.csv


In [15]:
has_duplicates = all_visit_data.duplicated().any()

if has_duplicates:
    print("row duplicated exist.")
else:
    print("row duplicated not exist.")

row duplicated not exist.


In [16]:
test_group_step = all_visit_data[all_visit_data['variation'] == 'Test']
test_group_step

,client_id,num_visits,variation,visitor_id,visit_id,start_count,step_1_count,step_2_count,step_3_count,confirm_count,last_process_step,date_ymd,last_time_stamp,last_step_confirm,step_hierarchy,if_confirm
49933,555,1,Test,402506806_56087378777,637149525_38041617439_716659,1,1,1,1,1,confirm,2017-04-15,13:00:34,1,confirm,1
49934,647,1,Test,66758770_53988066587,40369564_40101682850_311847,1,1,1,1,1,confirm,2017-04-12,15:47:45,1,confirm,1
49935,934,1,Test,810392784_45004760546,7076463_57954418406_971348,4,0,0,0,0,start,2017-04-18,02:38:52,0,start,0
24347,1336,2,Test,920624746_32603333901,614001770_19101025926_112779,0,0,0,0,2,confirm,2017-05-08,08:23:00,1,confirm,1
24348,1336,2,Test,920624746_32603333901,583743392_96265099036_939815,1,1,1,1,1,confirm,2017-05-08,06:08:43,1,confirm,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65173,9999626,1,Test,52633065_71189986073,182723364_27031318473_880901,1,1,0,0,0,step_1,2017-05-14,09:07:59,0,step_1,0
10483,9999729,3,Test,604429154_69247391147,99583652_41711450505_426179,1,1,0,0,0,step_1,2017-04-05,13:41:04,0,step_1,0
10484,9999729,3,Test,843385170_36953471821,493310979_9209676464_421146,2,1,1,0,0,start,2017-04-20,14:28:57,0,step_2,0
10482,9999729,3,Test,834634258_21862004160,870243567_56915814033_814203,1,1,1,1,1,confirm,2017-05-08,16:09:40,1,confirm,1


In [ ]:
test_group_step.nunique()

In [17]:
control_group_step = all_visit_data[all_visit_data['variation'] == 'Control']
control_group_step

,client_id,num_visits,variation,visitor_id,visit_id,start_count,step_1_count,step_2_count,step_3_count,confirm_count,last_process_step,date_ymd,last_time_stamp,last_step_confirm,step_hierarchy,if_confirm
49936,1028,1,Control,42237450_62128060588,557292053_87239438319_391157,1,5,2,1,0,step_1,2017-04-08,19:00:26,0,step_2,0
24344,1104,2,Control,194240915_18158000533,643221571_99977972121_69283,1,0,0,0,0,start,2017-06-20,22:31:33,0,start,0
24343,1104,2,Control,194240915_18158000533,543158812_46395476577_767725,1,0,0,0,0,start,2017-06-12,07:49:18,0,start,0
24346,1186,2,Control,446844663_31615102958,795373564_99931517312_810896,1,1,1,0,0,step_2,2017-04-08,18:05:24,0,step_2,0
24345,1186,2,Control,446844663_31615102958,507052512_11309370126_442139,1,0,0,0,0,start,2017-04-08,15:59:16,0,start,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10479,9997470,3,Control,395791369_55562604618,761490147_96352537762_21814,6,4,2,2,1,confirm,2017-05-09,16:39:34,1,confirm,1
10480,9997470,3,Control,91394485_75296404278,655572400_94971272893_411965,1,0,0,0,0,start,2017-04-07,16:11:03,0,start,0
65165,9998346,1,Control,292425655_16607136645,189177304_69869411700_783154,1,2,4,3,1,confirm,2017-03-29,15:37:28,1,confirm,1
65166,9998346,1,Control,292425655_16607136645,189177304_69869411700_783154,1,2,4,3,1,step_3,2017-03-29,15:37:28,0,confirm,1


In [ ]:
control_group_step.nunique()

In [18]:
all_visit_data.to_sql('all_visit_data',engine, 'project_ab_testing', if_exists='replace', index=False)
test_group_step.to_sql('test_group_step',engine, 'project_ab_testing', if_exists='replace', index=False)
control_group_step.to_sql('control_group_step',engine, 'project_ab_testing', if_exists='replace', index=False)

C:\Users\54189\AppData\Local\Temp\ipykernel_5792\3253850705.py:1: FutureWarning: Starting with pandas version 3.0 all arguments of to_sql except for the arguments 'name' and 'con' will be keyword-only.
  all_visit_data.to_sql('all_visit_data',engine, 'project_ab_testing', if_exists='replace', index=False)
C:\Users\54189\AppData\Local\Temp\ipykernel_5792\3253850705.py:2: FutureWarning: Starting with pandas version 3.0 all arguments of to_sql except for the arguments 'name' and 'con' will be keyword-only.
  test_group_step.to_sql('test_group_step',engine, 'project_ab_testing', if_exists='replace', index=False)
C:\Users\54189\AppData\Local\Temp\ipykernel_5792\3253850705.py:3: FutureWarning: Starting with pandas version 3.0 all arguments of to_sql except for the arguments 'name' and 'con' will be keyword-only.
  control_group_step.to_sql('control_group_step',engine, 'project_ab_testing', if_exists='replace', index=False)


32437

# Problems encountered:
- two different process_step with same time_stamp
- same visitor_id/visit_id for two different client_id


In [ ]:
# two different process_step with same time_stamp, ex
all_visit_data[all_visit_data['visit_id']=='377986493_6391607481_598681']

In [ ]:
# same visitor_id/visit_id for two different client_id, ex -> same variation
all_visit_data[all_visit_data['visit_id']=='30714723_53564046699_567312']

In [ ]:
# same visitor_id/visit_id for two different client_id, ex -> different variation
all_visit_data[all_visit_data['visit_id']=='92588242_2876965505_25554']

In [ ]:
visit_id_counts1 = each_step_count['visit_id'].value_counts()

duplicate_visit_ids1 = visit_id_counts1[visit_id_counts1 > 1].index.tolist()

for visit_id in duplicate_visit_ids1:
    num_occurrences1 = visit_id_counts1[visit_id]
    print(f"Visit ID: {visit_id}, Num Occurrences: {num_occurrences1}")


In [ ]:
visit_id_counts2 = last_process_step['visit_id'].value_counts()

duplicate_visit_ids2 = visit_id_counts2[visit_id_counts2 > 1].index.tolist()

for visit_id in duplicate_visit_ids2:
    num_occurrences2 = visit_id_counts2[visit_id]
    print(f"Visit ID: {visit_id}, Num Occurrences: {num_occurrences2}")

-  For bounce rate : how to define a "bounce" action ?
   - highest step reached 
   - last step before leaving (I used this one to calculate, but not sure)
-  Total number of visits : all visits or all clients ? (all visits maybe)

In [ ]:
test_group_step['last_process_step'].value_counts()

In [ ]:
test_group_step['step_hierarchy'].value_counts()

In [ ]:
control_group_step['last_process_step'].value_counts()

In [ ]:
control_group_step['step_hierarchy'].value_counts()

In [19]:
# number of clients
num_clients_test_group = test_group_step['client_id'].nunique()
print('Total number of clients for test group :', num_clients_test_group)
num_clients_control_group = control_group_step['client_id'].nunique()
print('Total number of clients for control group :', num_clients_control_group)

Total number of clients for test group : 26968
Total number of clients for control group : 23532


# Calculation of completion rate

In [20]:
total_num_visit_test_group = test_group_step['visit_id'].nunique()
print('Total number of visits for test group :',total_num_visit_test_group) 

Total number of visits for test group : 37136


In [23]:
total_num_visit_control_group = control_group_step['visit_id'].nunique()
print('Total number of visits for test group :',total_num_visit_control_group) 

Total number of visits for test group : 32189


In [24]:
# number of confirm for test group
confirm_count_test = test_group_step[test_group_step['if_confirm'] == 1]['if_confirm'].sum()
print("Confirm count for test group:", confirm_count_test)
# completion rate of test group
confirm_rate_test = (confirm_count_test / total_num_visit_test_group) * 100
print("Completion rate of test group:", round(confirm_rate_test,2), "%")

Confirm count for test group: 21820
Completion rate of test group: 58.76 %


In [25]:
# number of confirm for control group
confirm_count_control = control_group_step[control_group_step['if_confirm'] == 1]['if_confirm'].sum()
print("Confirm count for control group:", confirm_count_control)
# completion rate of control group
confirm_rate_control = (confirm_count_control / total_num_visit_control_group) * 100
print("Completion rate of control group:", round(confirm_rate_control,2), "%")

Confirm count for control group: 16268
Completion rate of control group: 50.54 %


# Calculation of "stop step" rate
- --> from which step the customers cannot go further (per visit)

In [26]:
# total count of each step in test group 
counts={}
steps = ['start', 'step_1', 'step_2', 'step_3']

for step in steps:
    counts[step]  = test_group_step[test_group_step['last_process_step'] == step].shape[0]
    print(f"{step.upper()} count in test group:", counts[step])

start_count_test = counts['start']
step_1_count_test = counts['step_1']
step_2_count_test = counts['step_2']
step_3_count_test = counts['step_3']

START count in test group: 9466
STEP_1 count in test group: 3154
STEP_2 count in test group: 1339
STEP_3 count in test group: 1811


In [27]:
# calculate bounce rate for each step in test group
stop_rates = {}

for step in steps:
    stop_rate = (counts[step] / total_num_visit_test_group) * 100
    stop_rates[step] = stop_rate

for step, stop_rate in stop_rates.items():
    print(f"Stop rate for the '{step}' step in test group:", round(stop_rate, 2), "%")

Stop rate for the 'start' step in test group: 25.49 %
Stop rate for the 'step_1' step in test group: 8.49 %
Stop rate for the 'step_2' step in test group: 3.61 %
Stop rate for the 'step_3' step in test group: 4.88 %


In [28]:
# total count of each step in control group 
counts2={}
steps2 = ['start', 'step_1', 'step_2', 'step_3']

for step in steps2:
    counts2[step]  = control_group_step[control_group_step['last_process_step'] == step].shape[0]
    print(f"{step.upper()} count in test group:", counts2[step])

start_count_control = counts2['start']
step_1_count_control = counts2['step_1']
step_2_count_control = counts2['step_2']
step_3_count_control = counts2['step_3']

START count in test group: 9652
STEP_1 count in test group: 3508
STEP_2 count in test group: 1469
STEP_3 count in test group: 2326


In [29]:
# calculate bounce rate for each step in control group
stop_rates2 = {}

for step in steps2:
    stop_rate2 = (counts2[step] / total_num_visit_control_group) * 100
    stop_rates2[step] = stop_rate2

for step, stop_rate in stop_rates2.items():
    print(f"Stop rate for the '{step}' step in control group:", round(stop_rate, 2), "%")

Stop rate for the 'start' step in control group: 29.99 %
Stop rate for the 'step_1' step in control group: 10.9 %
Stop rate for the 'step_2' step in control group: 4.56 %
Stop rate for the 'step_3' step in control group: 7.23 %


# Bounce rate
- --> the number of customers who only have "start" step (per visit)

In [31]:
all_visit_steps=pd.read_csv('all_visit_steps.csv')

In [32]:
all_visit_steps

,client_id,num_visits,variation,visitor_id,visit_id,start_count,step_1_count,step_2_count,step_3_count,confirm_count,last_process_step,date_ymd,last_time_stamp,last_step_confirm,step_hierarchy,if_confirm
0,555,1,Test,402506806_56087378777,637149525_38041617439_716659,1,1,1,1,1,confirm,2017-04-15,13:00:34,1,confirm,1
1,647,1,Test,66758770_53988066587,40369564_40101682850_311847,1,1,1,1,1,confirm,2017-04-12,15:47:45,1,confirm,1
2,934,1,Test,810392784_45004760546,7076463_57954418406_971348,4,0,0,0,0,start,2017-04-18,02:38:52,0,start,0
3,1028,1,Control,42237450_62128060588,557292053_87239438319_391157,1,5,2,1,0,step_1,2017-04-08,19:00:26,0,step_2,0
4,1104,2,Control,194240915_18158000533,643221571_99977972121_69283,1,0,0,0,0,start,2017-06-20,22:31:33,0,start,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69683,9999626,1,Test,52633065_71189986073,182723364_27031318473_880901,1,1,0,0,0,step_1,2017-05-14,09:07:59,0,step_1,0
69684,9999729,3,Test,604429154_69247391147,99583652_41711450505_426179,1,1,0,0,0,step_1,2017-04-05,13:41:04,0,step_1,0
69685,9999729,3,Test,843385170_36953471821,493310979_9209676464_421146,2,1,1,0,0,start,2017-04-20,14:28:57,0,step_2,0
69686,9999729,3,Test,834634258_21862004160,870243567_56915814033_814203,1,1,1,1,1,confirm,2017-05-08,16:09:40,1,confirm,1


In [33]:
columns_to_drop = ['visitor_id', 'last_step_confirm','step_hierarchy','if_confirm']
all_visit_steps = all_visit_steps.drop(columns=columns_to_drop)

In [34]:
all_visit_steps

,client_id,num_visits,variation,visit_id,start_count,step_1_count,step_2_count,step_3_count,confirm_count,last_process_step,date_ymd,last_time_stamp
0,555,1,Test,637149525_38041617439_716659,1,1,1,1,1,confirm,2017-04-15,13:00:34
1,647,1,Test,40369564_40101682850_311847,1,1,1,1,1,confirm,2017-04-12,15:47:45
2,934,1,Test,7076463_57954418406_971348,4,0,0,0,0,start,2017-04-18,02:38:52
3,1028,1,Control,557292053_87239438319_391157,1,5,2,1,0,step_1,2017-04-08,19:00:26
4,1104,2,Control,643221571_99977972121_69283,1,0,0,0,0,start,2017-06-20,22:31:33
...,...,...,...,...,...,...,...,...,...,...,...,...
69683,9999626,1,Test,182723364_27031318473_880901,1,1,0,0,0,step_1,2017-05-14,09:07:59
69684,9999729,3,Test,99583652_41711450505_426179,1,1,0,0,0,step_1,2017-04-05,13:41:04
69685,9999729,3,Test,493310979_9209676464_421146,2,1,1,0,0,start,2017-04-20,14:28:57
69686,9999729,3,Test,870243567_56915814033_814203,1,1,1,1,1,confirm,2017-05-08,16:09:40


In [36]:
summary_table=pd.read_csv('summary_table.csv')
summary_table

,client_id,study_group,highest_step_reached,step_hierarchy,completed_process,number_errors,encountered_errors,calls_6_mnth,gendr,logons_6_mnth,clnt_age,bal,clnt_tenure_yr,age_groups,longevity_groups,bal_groups,number_of_bounce
0,4192640,Test,step_2,3,No,2,Yes,3.0,M,6.0,54.0,97914.40,14.0,Middle-Aged Adults,Above-Average Longevity Clients,Moderate Balance,0.0
1,6752370,Test,step_2,3,No,2,Yes,2.0,M,5.0,22.0,56574.43,38.0,Young Adults,Exceptional Longevity Clients,Moderate Balance,0.0
2,2685910,Test,confirm,5,Yes,0,No,6.0,F,9.0,57.5,135363.65,12.0,Middle-Aged Adults,Above-Average Longevity Clients,Moderate Balance,0.0
3,3910683,Test,start,1,No,0,No,6.0,U,9.0,24.0,65233.30,5.0,Young Adults,Average Longevity Clients,Moderate Balance,2.0
4,2376452,Test,confirm,5,Yes,1,Yes,6.0,F,9.0,54.5,1073878.77,20.0,Middle-Aged Adults,Above-Average Longevity Clients,Very High Balance,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50495,8447700,Control,confirm,5,Yes,0,No,2.0,M,5.0,50.0,36227.90,7.0,Middle-Aged Adults,Average Longevity Clients,Low Balance,0.0
50496,2079465,Control,confirm,5,Yes,1,Yes,1.0,U,4.0,18.0,25022.78,4.0,Young Adults,Average Longevity Clients,Low Balance,0.0
50497,2061471,Control,confirm,5,Yes,0,No,1.0,F,4.0,51.0,36109.86,15.0,Middle-Aged Adults,Above-Average Longevity Clients,Low Balance,0.0
50498,1558312,Control,confirm,5,Yes,1,Yes,2.0,F,5.0,64.0,27918.86,23.0,Senior Adults,Exceptional Longevity Clients,Low Balance,0.0


In [37]:
df_merged = pd.merge(all_visit_steps, summary_table[['client_id', 'step_hierarchy']], on='client_id')

In [38]:
all_visit_steps=df_merged.copy()

In [39]:
# export test_group frame to CSV

# Get the current working directory
current_directory = os.getcwd()

# Specify the file name
file_name = "all_visit_steps_final.csv"

# Join the current directory with the file name to create the file path
file_path = os.path.join(current_directory, file_name)

# Export the summary_table DataFrame to a CSV file
all_visit_steps.to_csv(file_path, index=False)

print(f"Test group table has been exported to {file_path}")

Test group table has been exported to c:\Users\54189\ironhack_262\project2\project_ab_testing sql\all_visit_steps_final.csv


In [40]:
all_visit_steps=pd.read_csv('all_visit_steps_final.csv')

In [41]:
test_group_step = all_visit_steps[all_visit_steps['variation'] == 'Test']
control_group_step = all_visit_steps[all_visit_steps['variation'] == 'Control']

In [42]:
total_num_visit_test_group = test_group_step['visit_id'].nunique()
print('Total number of visits for test group :',total_num_visit_test_group) 
total_num_visit_control_group = control_group_step['visit_id'].nunique()
print('Total number of visits for test group :',total_num_visit_control_group) 

Total number of visits for test group : 37136
Total number of visits for test group : 32189


In [ ]:
test_group_step

In [43]:
only_start_test=test_group_step[test_group_step['step_hierarchy']==1]
num_bounce_test=len(only_start_test)
# print('Number of bounce for test group:',num_bounce_test)
bounce_rate_test = (num_bounce_test / total_num_visit_test_group) * 100
print("Bounce rate of test group:", round(bounce_rate_test,2), "%")

Bounce rate of test group: 7.48 %


In [44]:
only_start_control=control_group_step[control_group_step['step_hierarchy']==1]
num_bounce_control=len(only_start_control)
# print('Number of bounce for control group:',num_bounce_control)
bounce_rate_control = (num_bounce_control / total_num_visit_control_group) * 100
print("Bounce rate of control group:", round(bounce_rate_control,2), "%")

Bounce rate of control group: 11.26 %
